# Question 2

## Weak Form

The weak form of the Euler-Bernoulli beam element is given by:
\begin{align*}
0 = &\int_{x_e}^{x_{e+1}} \left(
    EI \frac{d^2 v}{dx^2}\frac{d^2 w}{dx^2} + c_f v w - v q 
\right) dx\\
&- v(x_e)Q_1^e - \left(-\frac{dv}{dx}\right)\big|_{x_e}Q_2^e
- v(x_{e+1})Q_3^e - \left(-\frac{dv}{dx}\right)\big|_{x_{e+1}}Q_4^e
\end{align*}

### a)

We consider a three-node element with two degrees of freedom ($w, \theta$) where $\theta = -dw/dx$. It is assumed that the interior 
node is equidistant from the outer nodes, i.e. :
$$
x_{interior} = x_{e+1/2} = \frac{x_e + x_{e+1}}{2}
$$

The approximation $w_h^e(x)$ of $w(x)$ over our element should
satisfy the following interpolation properties:
\begin{align*}
&w_h^e(x_e) = \Delta_1^e = w_1^e, \quad 
    &&\Delta_2^e = \theta_h^e(x_e) = \theta_1^e\\
&w_h^e(x_{e+1/2}) = \Delta_3^e = w_2^e, \quad 
    &&\Delta_4^e = \theta_h^e(x_{e+1/2}) = \theta_2^e\\
&w_h^e(x_{e+1}) = \Delta_5^e = w_3^e, \quad 
    &&\Delta_6^e = \theta_h^e(x_{e+1}) = \theta_3^e
\end{align*}
To satisfy all conditions, we need a six-parameter polynomial:
$$
w(x) \approx w_h^e(x) = c_1^e + c_2^e x + c_3^e x^2 + c_4^e x^3
    + c_5^e x^4 + c_6^e x^5
$$
The next step involves expressing $c_i^e$ in terms of $\Delta_j^e$ -- this is where we begin to use sympy. 

In [1]:
import sympy as sp
from sympy.matrices.expressions.matmul import MatMul
import numpy as np
sp.init_printing(num_columns=140)

from sympyutils import show, show_list, show_add

In [2]:
all_c = sp.symbols('c1:7', real=True)
# all_c = sp.symbols('c1:5', real=True)
num_c = len(all_c)
x = sp.symbols('x', real=True)
h = sp.symbols('h_e', real=True)
all_xe = sp.symbols('x_e x_{e+1/2} x_{e+1}', real=True)
# all_xe = sp.symbols('x_e x_{e+1}')
all_xe

In [3]:
w = sum( [all_c[i]*x**i for i in range(num_c)] )
show(w, 'w_h^e(x)')

<IPython.core.display.Math object>

In [4]:
t = -sp.diff(w, x)
show(t, r'\theta_h^e(x)')

<IPython.core.display.Math object>

In [5]:
all_d = sp.symbols('Delta_1:7')
# all_d = sp.symbols('Delta_1:5')
all_d

In [6]:
d_eqs = []
for i, xe in enumerate(all_xe):
    d1 = w.subs(x, xe)
    d_eqs.append(d1)
    d2 = t.subs(x, xe)
    d_eqs.append(d2)
d_names = [sp.latex(d) for d in all_d]
show_list(d_eqs, d_names)

<IPython.core.display.Math object>

In [7]:
def pop_A(i, j):
    di = d_eqs[i]
    return di.coeff(all_c[j])

In [8]:
A = sp.Matrix(num_c, num_c, pop_A)
A = A.subs(all_xe[1], all_xe[0] + h/2).subs(all_xe[2], all_xe[0] + h)
# A = A.subs(all_xe[1], all_xe[0] + h)
# Switch to local coordinates
A = A.subs(all_xe[0], 0)

show(A, 'A')

<IPython.core.display.Math object>

In [9]:
D = sp.Matrix(all_d)
show(D, 'D')

<IPython.core.display.Math object>

In [10]:
C = A.inv().dot(D)

In [11]:
xbar = sp.symbols(r'\bar{x}')

In [12]:
ww = w.subs( [(all_c[i], C[i]) for i in range(num_c)] ).subs(x, xbar)
ww = ww.simplify().expand()

In [13]:
# Some magic to extract the terms
all_phi = []
for d in all_d:
    coeff = ww.coeff(d).simplify().expand()
    all_phi.append(coeff)
all_d2phi = [sp.diff(phi, xbar, 2).simplify().expand() for phi in all_phi]

In [14]:
num_phi = len(all_phi)
phi_names = [r'\phi_%1i' % (i+1) for i in range(num_phi)]
d2phi_names = [r'\frac{d^2\phi_%i}{dx^2}' % (i+1) for i in range(num_phi)]

In [15]:
show_list(all_phi, phi_names)

<IPython.core.display.Math object>

In [16]:
show_list(all_d2phi, d2phi_names)

<IPython.core.display.Math object>

*Q.E.D.*

---

### b)

The cells of the element stiffness $[K^e]$ are given by:
$$
K^e_{ij} = \int_{0}^{h_e} \left( 
    EI \frac{d^2\phi_i^e}{d\bar{x}^2}\frac{d^2\phi_j^e}{d\bar{x}^2}
    + c_f\phi_i^e\phi_j^e
\right) d\bar{x}
$$
and those of the force vector $\{F^e\}$ by:
$$
F_i^e = \int_{0}^{h_e} \phi_i^e q d\bar{x} + Q_i^e
$$
where it is assumed in this case that $q, c_f, E, I$ are constants, i.e. not 
functions of $x$.

In [17]:
EI, q, cf = sp.symbols('EI q c_f')
def pop_K1(i, j):
    k1 = EI*sp.integrate(all_d2phi[i]*all_d2phi[j], (xbar, 0, h)).simplify()
    return k1
def pop_K2(i, j):
    k2 = cf*sp.integrate(all_phi[i]*all_phi[j], (xbar, 0, h)).simplify()
    return k2

def pop_F(i, j):
    return q*sp.integrate(all_phi[i], (xbar, 0, h)).simplify()

In [18]:
K1 = sp.Matrix(num_phi, num_phi, pop_K1)
K2 = sp.Matrix(num_phi, num_phi, pop_K2)
F = sp.Matrix(num_phi, 1, pop_F)

In [19]:
# Take common factor out (obtained by inspection)
prediv1 = EI/(35*h**3)
K1 = MatMul(prediv1, K1/prediv1)
prediv2 = cf*h/(13860)
K2 = MatMul(prediv2, K2/prediv2)
predivF = h*q/60
F = MatMul(predivF, F/predivF)

In [20]:
show_add([K1, K2], 'K^e')

<IPython.core.display.Math object>

In [21]:
show(F, 'F^e')

<IPython.core.display.Math object>